In [ ]:
project_name = "sample"
average_volume_file = "../data/sample.mrc"
alignment_file = "../data/sample.star"
image_stack_file = "../data/sample.mrcs"
pixel_size = 1.255  
diameter = 360 
resolution = 5 
aperture_index = 4  

In [ ]:
import numpy as np
import mrcfile
import shutil
import os
from params import p

In [ ]:
def get_image_width_from_stack(stack_file: str):
    """
    Determines the width of images in a file stack.

    Parameters:
    - stack_file (str): The path to the stack file.

    Returns:
    - int: The width of the images contained in the stack file.

    Notes:
    - For MRC stack files (.mrcs), the function directly reads the file using the mrcfile library
      to determine the image width.
    - For other file formats, it estimates the image width based on the file size and the number
      of particles (`p.num_part`), assuming each pixel is stored as a 4-byte float.
    - This function assumes that the images are square and that the total file size can be divided
      by the product of the number of particles, the square of the image width, and the size of
      each pixel (4 bytes for a float).
    """
    img_width = 0  # Initialize image width to 0

    if stack_file.endswith('.mrcs'):
        # Handle MRC stack files
        mrc = mrcfile.mmap(stack_file, mode='r')  # Memory-map the file in read-only mode
        if not mrc.is_image_stack():
            # If the file is not recognized as an image stack, try to set it as one
            mrc.close()  # Close the current mmap
            mrc = mrcfile.mmap(stack_file, mode='r+')  # Re-open the file in read-write mode
            mrc.set_image_stack()  # Attempt to set the file as an image stack
        img_width = mrc.data[0].shape[0]  # Get the width of the first image in the stack
        mrc.close()  # Close the file
    else:
        # Handle other file formats by estimating the image width
        file_size = os.path.getsize(stack_file)  # Get the file size in bytes
        img_width = int(np.sqrt(file_size / (4 * p.num_part)))  # Estimate the image width

    return img_width

In [ ]:
def init_project(project_name, average_volume_file, alignment_file, image_stack_file, pixel_size, 
                 diameter, resolution, aperture_index, overwrite=False):
    """
    Initializes a new project with specified parameters, including project directory creation
    and parameter storage.

    Parameters:
    - project_name (str): The name of the project.
    - average_volume_file (str): Path to the file containing the average volume data.
    - alignment_file (str): Path to the file containing alignment parameters.
    - image_stack_file (str): Path to the file containing the image stack.
    - pixel_size (float): The pixel size in the images.
    - diameter (float): The diameter of the object in the images.
    - resolution (float): The resolution estimate for the images.
    - aperture_index (int): The index of the aperture used.
    - overwrite (bool, optional): Whether to overwrite an existing project with the same name.
      Defaults to False.

    Notes:
    - The function sets project parameters in a global variable `p`, which is assumed to be an
      instance of a class with attributes corresponding to the project parameters and methods
      for directory creation (`create_dir`) and parameter storage (`save`).
    - It checks if the project directory already exists and handles it based on the `overwrite`
      option. If `overwrite` is True, the existing directory is removed; otherwise, the function
      exits without creating a new project.
    - The function determines whether the alignment file is in RELION's STAR format based on its
      file extension and sets a flag accordingly.
    - The image width is determined from the image stack file using the `get_image_width_from_stack`
      function, which is assumed to be defined elsewhere.
    - Project parameters are saved to a TOML file using the `save` method of the global variable `p`.
    """
    # Set project parameters
    p.proj_name = project_name
    p.avg_vol_file = average_volume_file
    p.align_param_file = alignment_file
    p.img_stack_file = image_stack_file
    p.pix_size = pixel_size
    p.obj_diam = diameter
    p.resol_est = resolution
    p.ap_index = aperture_index
    p.relion_data = alignment_file.endswith('.star')  # Check if alignment file is in RELION STAR format
    
    # Determine image width from stack
    p.nPix = get_image_width_from_stack(image_stack_file)
    
    # Check if project directory exists and handle overwrite option
    if os.path.exists(p.out_dir):
        if overwrite:
            print(f"Overwriting existing project: {project_name}")
            shutil.rmtree(p.out_dir)  # Remove existing project directory
        else:
            print(f"Project {project_name} already exists. Use overwrite=True to overwrite.")
            return  # Exit if not overwriting
    
    # Create project directories
    p.create_dir()
    
    # Save project parameters to TOML file
    p.save()

In [ ]:
init_project(project_name, average_volume_file, alignment_file, image_stack_file, pixel_size, diameter, 
             resolution, aperture_index, overwrite=True)